In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset

# First, let's load the dataset and check what splits are available
dataset = load_dataset("klue", "nli")
print(f"Available splits in the dataset: {list(dataset.keys())}")

# Define your custom classification model
class OLMoForSequenceClassification(torch.nn.Module):
    def __init__(self, model_path, num_labels=3):
        super().__init__()
        # Load OLMo Causal LM model
        self.olmo = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            trust_remote_code=True
        )
        # Add classification head
        self.classifier = torch.nn.Linear(
            self.olmo.config.hidden_size,  # OLMo's hidden size
            num_labels                     # KLUE-NLI class count (0: entail, 1: neutral, 2: contradiction)
        )
        
        # Move classifier to same device as model
        device = next(self.olmo.parameters()).device
        self.classifier = self.classifier.to(device)
    
    def forward(self, input_ids, attention_mask):
        # Extract OLMo outputs
        outputs = self.olmo(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        # Extract [CLS] token from last hidden state
        last_hidden_state = outputs.hidden_states[-1]
        cls_token = last_hidden_state[:, 0, :]  # Use first token as [CLS]
        # Run classification
        logits = self.classifier(cls_token)
        return logits

Available splits in the dataset: ['train', 'validation']


In [5]:
# Load tokenizer
model_path = "fine-tuned-models/fine-tuned-olmo1B-80000-v11"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Create your custom model
model = OLMoForSequenceClassification(model_path, num_labels=3)

# Use validation split if test is not available
test_dataset = dataset["validation"]  # Changed from "test" to "validation"

# Data preprocessing function
def preprocess(examples):
    inputs = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors=None  # Changed to None for batched processing
    )
    inputs["label"] = examples["label"]
    return inputs

test_dataset = test_dataset.map(preprocess, batched=True)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [6]:
# Adjust the dataset format for compatibility with your custom model
from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["label"] for item in batch])
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

test_dataloader = DataLoader(
    test_dataset, 
    batch_size=32,  # Reduced batch size to save memory
    collate_fn=collate_fn
)

# Evaluation
device = next(model.parameters()).device  # Get model device
model.eval()
predictions = []
labels = []

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    
    with torch.no_grad():
        outputs = model(batch["input_ids"], batch["attention_mask"])
    
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    batch_labels = batch["labels"].cpu().numpy()
    
    predictions.extend(preds)
    labels.extend(batch_labels)

In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(labels, predictions)
print(f"Test Accuracy: {accuracy:.4f}")

# Save prediction results
import pandas as pd
df = pd.DataFrame({"pred_label": predictions})
df.to_csv("klue_nli_olmo1b_nli.csv", index=False)

Test Accuracy: 0.3333


In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd

# 원본 OLMo 모델 경로
original_model_path = "allenai/OLMo-7B-hf"

# 커스텀 분류 모델 정의
class OLMoForSequenceClassification(torch.nn.Module):
    def __init__(self, model_path, num_labels=3):
        super().__init__()
        # OLMo Causal LM 모델 로드
        self.olmo = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            trust_remote_code=True
        )
        # 분류 헤드 추가
        self.classifier = torch.nn.Linear(
            self.olmo.config.hidden_size,  # OLMo의 hidden size
            num_labels                     # KLUE-NLI 클래스 수 (0: entail, 1: neutral, 2: contradiction)
        )
        
        # 분류기를 모델과 같은 장치로 이동
        device = next(self.olmo.parameters()).device
        self.classifier = self.classifier.to(device)
    
    def forward(self, input_ids, attention_mask):
        # OLMo 출력 추출
        outputs = self.olmo(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        # 마지막 hidden state에서 [CLS] 토큰 추출
        last_hidden_state = outputs.hidden_states[-1]
        cls_token = last_hidden_state[:, 0, :]  # 첫 번째 토큰을 [CLS]로 사용
        # 분류 실행
        logits = self.classifier(cls_token)
        return logits

# 데이터셋 로드
print("데이터셋 로드 중...")
dataset = load_dataset("klue", "nli")
print(f"사용 가능한 데이터셋 스플릿: {list(dataset.keys())}")
test_dataset = dataset["validation"]

# 오리지널 OLMo 토크나이저 로드
print("토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(original_model_path, trust_remote_code=True)

# 데이터 전처리 함수
def preprocess(examples):
    # Add explicit separator between premise and hypothesis
    text_pairs = [
        f"Premise: {premise} Hypothesis: {hypothesis}" 
        for premise, hypothesis in zip(examples["premise"], examples["hypothesis"])
    ]
    
    inputs = tokenizer(
        text_pairs,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors=None
    )
    inputs["label"] = examples["label"]
    return inputs

print("데이터셋 전처리 중...")
test_dataset = test_dataset.map(preprocess, batched=True)

# 테스트용 작은 데이터셋 생성 (선택 사항)
# test_dataset = test_dataset.select(range(100))  # 처음 100개 샘플만 사용

# 데이터로더 설정
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["label"] for item in batch])
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# 오리지널 OLMo 모델로 분류 모델 생성
print("원본 OLMo 모델 로드 중...")
model = OLMoForSequenceClassification(original_model_path, num_labels=3)

# GPU 메모리가 충분하므로 배치 크기 증가
test_dataloader = DataLoader(
    test_dataset, 
    batch_size=32,  # 메모리 사용량에 따라 조정
    collate_fn=collate_fn
)

# 평가
print("평가 시작...")
device = next(model.parameters()).device
model.eval()
predictions = []
labels = []

total_batches = len(test_dataloader)
for i, batch in enumerate(test_dataloader):
    if i % 10 == 0:  # 진행 상황 표시
        print(f"배치 처리 중: {i}/{total_batches} ({i/total_batches*100:.2f}%)")
    
    batch = {k: v.to(device) for k, v in batch.items()}
    
    with torch.no_grad():
        outputs = model(batch["input_ids"], batch["attention_mask"])
    
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    batch_labels = batch["labels"].cpu().numpy()
    
    predictions.extend(preds)
    labels.extend(batch_labels)

# 결과 평가
accuracy = accuracy_score(labels, predictions)
print(f"원본 OLMo 모델 테스트 정확도: {accuracy:.4f}")

# 결과 저장
df = pd.DataFrame({"pred_label": predictions})
df.to_csv("klue_nli_olmo7b_nli.csv", index=False)

print("평가 완료!")

데이터셋 로드 중...
사용 가능한 데이터셋 스플릿: ['train', 'validation']
토크나이저 로드 중...
데이터셋 전처리 중...


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

원본 OLMo 모델 로드 중...


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


평가 시작...
배치 처리 중: 0/94 (0.00%)
배치 처리 중: 10/94 (10.64%)
배치 처리 중: 20/94 (21.28%)
배치 처리 중: 30/94 (31.91%)
배치 처리 중: 40/94 (42.55%)
배치 처리 중: 50/94 (53.19%)
배치 처리 중: 60/94 (63.83%)
배치 처리 중: 70/94 (74.47%)
배치 처리 중: 80/94 (85.11%)
배치 처리 중: 90/94 (95.74%)
원본 OLMo 모델 테스트 정확도: 0.3333
평가 완료!


In [14]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd
import ollama

# Ollama 클라이언트 설정
ollama_host = "http://sg011:11434"
client = ollama.Client(host=ollama_host)

# Change the default embedding_size from 4096 to 3072
class OllamaForSequenceClassification(torch.nn.Module):
    def __init__(self, model_name, num_labels=3):
        super().__init__()
        self.model_name = model_name
        self.client = ollama.Client(host=ollama_host)
        
        # Dynamically determine embedding size from first sample
        sample_text = "This is a test."
        response = self.client.embeddings(model=self.model_name, prompt=sample_text)
        self.embedding_size = len(response['embedding'])
        print(f"Detected embedding size: {self.embedding_size}")
        
        # Use detected size for classifier
        self.classifier = torch.nn.Linear(self.embedding_size, num_labels)
        
        # GPU setup
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.classifier = self.classifier.to(self.device)
        
    def get_embeddings(self, texts):
        """Ollama API를 사용하여 텍스트 임베딩 획득"""
        embeddings = []
        for text in texts:
            response = self.client.embeddings(model=self.model_name, prompt=text)
            embedding = torch.tensor(response['embedding'])
            embeddings.append(embedding)
        return torch.stack(embeddings).to(self.device)
    
    def forward(self, input_texts):
        """텍스트 입력을 받아 분류 수행"""
        embeddings = self.get_embeddings(input_texts)
        logits = self.classifier(embeddings)
        return logits

# 데이터셋 로드
print("데이터셋 로드 중...")
dataset = load_dataset("klue", "nli")
print(f"사용 가능한 데이터셋 스플릿: {list(dataset.keys())}")
test_dataset = dataset["validation"]

# 데이터 전처리 함수 (텍스트 연결)
def preprocess(examples):
    # premise와 hypothesis를 하나의 텍스트로 결합
    combined_texts = [f"Premise: {p} Hypothesis: {h}" for p, h in zip(examples["premise"], examples["hypothesis"])]
    return {
        "combined_text": combined_texts,
        "label": examples["label"]
    }

print("데이터셋 전처리 중...")
test_dataset = test_dataset.map(preprocess, batched=True)

# 데이터로더 설정
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "label": self.labels[idx]
        }

text_dataset = TextDataset(test_dataset["combined_text"], test_dataset["label"])

# 배치 사이즈는 작게 설정 (API 호출 횟수 고려)
test_dataloader = DataLoader(
    text_dataset, 
    batch_size=4,  # API 호출이므로 작은 배치 크기 사용
    shuffle=False
)

# Ollama 모델로 분류 모델 생성
print("Ollama Llama 3.2 모델 준비 중...")
model = OllamaForSequenceClassification("llama3.2", num_labels=3)

# 평가
print("평가 시작...")
model.eval()
predictions = []
labels = []

total_batches = len(test_dataloader)
for i, batch in enumerate(test_dataloader):
    print(f"배치 처리 중: {i+1}/{total_batches} ({(i+1)/total_batches*100:.2f}%)")
    
    texts = batch["text"]
    batch_labels = batch["label"].numpy()
    
    with torch.no_grad():
        outputs = model(texts)
    
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    
    predictions.extend(preds)
    labels.extend(batch_labels)

# 결과 평가
accuracy = accuracy_score(labels, predictions)
print(f"Ollama Llama 3.2 모델 테스트 정확도: {accuracy:.4f}")

# 결과 저장
df = pd.DataFrame({"pred_label": predictions})
df.to_csv("ollama_llama3_2_predictions.csv", index=False)

print("평가 완료!")

데이터셋 로드 중...
사용 가능한 데이터셋 스플릿: ['train', 'validation']
데이터셋 전처리 중...


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Ollama Llama 3.2 모델 준비 중...
Detected embedding size: 3072
평가 시작...
배치 처리 중: 1/750 (0.13%)
배치 처리 중: 2/750 (0.27%)
배치 처리 중: 3/750 (0.40%)
배치 처리 중: 4/750 (0.53%)
배치 처리 중: 5/750 (0.67%)
배치 처리 중: 6/750 (0.80%)
배치 처리 중: 7/750 (0.93%)
배치 처리 중: 8/750 (1.07%)
배치 처리 중: 9/750 (1.20%)
배치 처리 중: 10/750 (1.33%)
배치 처리 중: 11/750 (1.47%)
배치 처리 중: 12/750 (1.60%)
배치 처리 중: 13/750 (1.73%)
배치 처리 중: 14/750 (1.87%)
배치 처리 중: 15/750 (2.00%)
배치 처리 중: 16/750 (2.13%)
배치 처리 중: 17/750 (2.27%)
배치 처리 중: 18/750 (2.40%)
배치 처리 중: 19/750 (2.53%)
배치 처리 중: 20/750 (2.67%)
배치 처리 중: 21/750 (2.80%)
배치 처리 중: 22/750 (2.93%)
배치 처리 중: 23/750 (3.07%)
배치 처리 중: 24/750 (3.20%)
배치 처리 중: 25/750 (3.33%)
배치 처리 중: 26/750 (3.47%)
배치 처리 중: 27/750 (3.60%)
배치 처리 중: 28/750 (3.73%)
배치 처리 중: 29/750 (3.87%)
배치 처리 중: 30/750 (4.00%)
배치 처리 중: 31/750 (4.13%)
배치 처리 중: 32/750 (4.27%)
배치 처리 중: 33/750 (4.40%)
배치 처리 중: 34/750 (4.53%)
배치 처리 중: 35/750 (4.67%)
배치 처리 중: 36/750 (4.80%)
배치 처리 중: 37/750 (4.93%)
배치 처리 중: 38/750 (5.07%)
배치 처리 중: 39/750 (5.20%)
배치 처리 